<center><h1>DenseNet: Cifar10</h1></center>

## Imports

In [0]:
from __future__ import division,print_function

%matplotlib inline
%reload_ext autoreload
%autoreload 2

import sys
from tqdm import tqdm_notebook as tqdm

import random
import matplotlib.pyplot as plt
import math

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from models import densenet, new_densenet
from torch.autograd import Variable, grad
from torchvision import datasets, transforms
from torch.nn.parameter import Parameter
from PIL import Image as PILImage
import zero_shot_calculate_log as callog

import warnings
warnings.filterwarnings('ignore')

In [0]:
torch.cuda.set_device(1) #Select the GPU

## Model definition

In [0]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

def G_p(ob, p):
    temp = ob.detach()

    temp = temp**p
    temp = temp.reshape(temp.shape[0],temp.shape[1],-1)
    temp = ((torch.matmul(temp,temp.transpose(dim0=2,dim1=1)))).sum(dim=2)
    temp = (temp.sign()*torch.abs(temp)**(1/p)).reshape(temp.shape[0],-1)

    return temp


class BottleneckBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(BottleneckBlock, self).__init__()
        inter_planes = out_planes * 4
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, inter_planes, kernel_size=1, stride=1,
                               padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(inter_planes)
        self.conv2 = nn.Conv2d(inter_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
    
    def forward(self, x):
        
        out = self.conv1(self.relu(self.bn1(x)))
        
        torch_model.record(out)
        
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        
        out = self.conv2(self.relu(self.bn2(out)))
        torch_model.record(out)
        
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        return torch.cat([x, out], 1)

class TransitionBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(TransitionBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1,
                               padding=0, bias=False)
        self.droprate = dropRate
    
    def forward(self, x):
        out = self.conv1(self.relu(self.bn1(x)))
        torch_model.record(out)
        
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        return F.avg_pool2d(out, 2)

class DenseBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, growth_rate, block, dropRate=0.0):
        super(DenseBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, growth_rate, nb_layers, dropRate)
    
    def _make_layer(self, block, in_planes, growth_rate, nb_layers, dropRate):
        layers = []
        for i in range(int(nb_layers)):
            layers.append(block(in_planes+i*growth_rate, growth_rate, dropRate))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        t = self.layer(x)
        torch_model.record(t)
        return t


class DenseNet3(nn.Module):
    def __init__(self, depth, num_classes, growth_rate=12,
                 reduction=0.5, bottleneck=True, dropRate=0.0):
        super(DenseNet3, self).__init__()
        
        self.collecting = False
        
        in_planes = 2 * growth_rate
        n = (depth - 4) / 3
        if bottleneck == True:
            n = n/2
            block = BottleneckBlock
        else:
            block = BasicBlock
        # 1st conv before any dense block
        self.conv1 = nn.Conv2d(3, in_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = DenseBlock(n, in_planes, growth_rate, block, dropRate)
        in_planes = int(in_planes+n*growth_rate)
        self.trans1 = TransitionBlock(in_planes, int(math.floor(in_planes*reduction)), dropRate=dropRate)
        in_planes = int(math.floor(in_planes*reduction))
        # 2nd block
        self.block2 = DenseBlock(n, in_planes, growth_rate, block, dropRate)
        in_planes = int(in_planes+n*growth_rate)
        self.trans2 = TransitionBlock(in_planes, int(math.floor(in_planes*reduction)), dropRate=dropRate)
        in_planes = int(math.floor(in_planes*reduction))
        # 3rd block
        self.block3 = DenseBlock(n, in_planes, growth_rate, block, dropRate)
        in_planes = int(in_planes+n*growth_rate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(in_planes, num_classes)
        self.in_planes = in_planes

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()
                    
    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.block1(out))
        out = self.trans2(self.block2(out))
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(-1, self.in_planes)
        return self.fc(out)
    
    def load(self, path="./pre_trained/densenet_cifar10.pth"):
        tm = torch.load(path, map_location="cpu")
#         self.load_state_dict(tm.state_dict(),strict=False)
        self.load_state_dict(tm)
    
    def record(self, t):
        if self.collecting:
            self.gram_feats.append(t)
    
    def gram_feature_list(self,x):
        self.collecting = True
        self.gram_feats = []
        self.forward(x)
        self.collecting = False
        temp = self.gram_feats
        self.gram_feats = []
        return temp
    
    def get_min_max(self, data, power):
        mins = []
        maxs = []
        
        for i in range(0,len(data),64):
            batch = data[i:i+64].cuda()
            feat_list = self.gram_feature_list(batch)
            for L,feat_L in enumerate(feat_list):
                if L==len(mins):
                    mins.append([None]*len(power))
                    maxs.append([None]*len(power))
                
                for p,P in enumerate(power):
                    g_p = G_p(feat_L,P)
                    
                    current_min = g_p.min(dim=0,keepdim=True)[0]
                    current_max = g_p.max(dim=0,keepdim=True)[0]
                    
                    if mins[L][p] is None:
                        mins[L][p] = current_min
                        maxs[L][p] = current_max
                    else:
                        mins[L][p] = torch.min(current_min,mins[L][p])
                        maxs[L][p] = torch.max(current_max,maxs[L][p])
        
        return mins,maxs
    
    def get_deviations(self,data,power,mins,maxs):
        deviations = []
        
        for i in range(0,len(data),64):            
            batch = data[i:i+64].cuda()
            feat_list = self.gram_feature_list(batch)
            batch_deviations = []
            for L,feat_L in enumerate(feat_list):
                dev = 0
                for p,P in enumerate(power):
                    g_p = G_p(feat_L,P)
                    
                    dev +=  (F.relu(mins[L][p]-g_p)/torch.abs(mins[L][p]+10**-6)).sum(dim=1,keepdim=True)
                    dev +=  (F.relu(g_p-maxs[L][p])/torch.abs(maxs[L][p]+10**-6)).sum(dim=1,keepdim=True)
                batch_deviations.append(dev.cpu().detach().numpy())
            batch_deviations = np.concatenate(batch_deviations,axis=1)
            deviations.append(batch_deviations)
        deviations = np.concatenate(deviations,axis=0)
        
        return deviations

torch_model = DenseNet3(100, num_classes=10)
# torch_model = torch.nn.DataParallel(torch_model).cuda()
model = torch.load('./results/Zero_Shot/Zero_Shot_cifar10_densenet_oe_tune_epoch_14.pth')
torch_model.load_state_dict(model), #strict=False) 
# torch_model= torch_model.module
# torch_model.load()
torch_model.cuda()
torch_model.params = list(torch_model.parameters())
torch_model.eval()
print("Done")  

Done


## Datasets

<b>In-distribution Datasets</b>

In [0]:
batch_size = 128
mean = np.array([[125.3/255, 123.0/255, 113.9/255]]).T

std = np.array([[63.0/255, 62.1/255.0, 66.7/255.0]]).T
normalize = transforms.Normalize((125.3/255, 123.0/255, 113.9/255), (63.0/255, 62.1/255.0, 66.7/255.0))

transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
        
    ])
transform_test = transforms.Compose([
#     transforms.CenterCrop(size=(32, 32)),
        transforms.ToTensor(),
        normalize
    ])

train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data/cifar10-data', train=True, download=True,
                   transform=transform_train),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data/cifar10-data', train=False, transform=transform_test),
    batch_size=batch_size)

data_train = list(torch.utils.data.DataLoader(
        datasets.CIFAR10('data/cifar10-data', train=True, download=True,
                       transform=transform_test),
        batch_size=1, shuffle=False))

data = list(torch.utils.data.DataLoader(
    datasets.CIFAR10('data/cifar10-data', train=False, download=True,
                   transform=transform_test),
    batch_size=1, shuffle=False))

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [0]:
torch_model.eval()
correct = 0
total = 0
for x,y in test_loader:
    x = x.cuda()
    y = y.numpy()
    correct += (y==np.argmax(torch_model(x).detach().cpu().numpy(),axis=1)).sum()
    total += y.shape[0]
print("Accuracy: ",correct/total)


Accuracy:  0.9519


<b>Out-of-distribution Datasets</b>

In [0]:
batch_size = 1

In [0]:
svhn = list(torch.utils.data.DataLoader(
    datasets.SVHN('data/svhn-data', split="test", download=True,
                   transform=transform_test),
    batch_size=batch_size, shuffle=True))

Using downloaded and verified file: data/svhn-data/test_32x32.mat


In [0]:
#isun = list(torch.utils.data.DataLoader(
    #datasets.ImageFolder("iSUN/",transform=transform_test),batch_size=1,shuffle=True))

In [0]:
lsun = list(torch.utils.data.DataLoader(
    datasets.ImageFolder("data/LSUN_resize/",transform=transform_test),batch_size=batch_size,shuffle=True))

In [0]:
tinyimagenet = list(torch.utils.data.DataLoader(
    datasets.ImageFolder("data/Imagenet_resize/",transform=transform_test),batch_size=batch_size,shuffle=True))

<b>Validation -> Out-of-distribution Datasets</b>

In [0]:
# /////////////// Uniform Noise ///////////////
ood_num_examples = len(data)//5

dummy_targets = torch.ones(ood_num_examples)
ood_data = torch.from_numpy(
    np.random.uniform(size=(ood_num_examples, 3, 32, 32),
                      low=-1.0, high=1.0).astype(np.float32))
ood_data = torch.utils.data.TensorDataset(ood_data, dummy_targets)
uniform_loader = list(torch.utils.data.DataLoader(ood_data, batch_size=1, shuffle=True))

In [0]:
# /////////////// Arithmetic Mean of Images ///////////////

ood_data = datasets.CIFAR10('data/cifar10-data', download=True,train=False, transform=transform_test)


class AvgOfPair(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.shuffle_indices = np.arange(len(dataset))
        np.random.shuffle(self.shuffle_indices)

    def __getitem__(self, i):
        random_idx = np.random.choice(len(self.dataset))
        while random_idx == i:
            random_idx = np.random.choice(len(self.dataset))

        return self.dataset[i][0] / 2. + self.dataset[random_idx][0] / 2., 0

    def __len__(self):
        return len(self.dataset)


ami_loader = list(torch.utils.data.DataLoader(AvgOfPair(ood_data),
                                         batch_size=batch_size, shuffle=True,pin_memory=True))

Files already downloaded and verified


In [0]:
# /////////////// Geometric Mean of Images ///////////////
ood_data = datasets.CIFAR10('data/cifar10-data', download=True,train=False, transform=transform_test)

class GeomMeanOfPair(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.shuffle_indices = np.arange(len(dataset))
        np.random.shuffle(self.shuffle_indices)

    def __getitem__(self, i):
        random_idx = np.random.choice(len(self.dataset))
        while random_idx == i:
            random_idx = np.random.choice(len(self.dataset))

        return normalize(torch.sqrt(self.dataset[i][0] * self.dataset[random_idx][0])), 0

    def __len__(self):
        return len(self.dataset)


gmi_loader = list(torch.utils.data.DataLoader(
    GeomMeanOfPair(ood_data), batch_size=batch_size, shuffle=True,pin_memory=True))

Files already downloaded and verified


In [0]:
# /////////////// Jigsaw Images ///////////////

ood_loader = torch.utils.data.DataLoader(ood_data, batch_size=batch_size, shuffle=True,
                                         num_workers=4, pin_memory=True)

jigsaw = lambda x: torch.cat((
    torch.cat((torch.cat((x[:, 8:16, :16], x[:, :8, :16]), 1),
               x[:, 16:, :16]), 2),
    torch.cat((x[:, 16:, 16:],
               torch.cat((x[:, :16, 24:], x[:, :16, 16:24]), 2)), 2),
), 1)

ood_loader.dataset.transform = transforms.Compose([transforms.ToTensor(), jigsaw, normalize])
jigsaw_loader = list(ood_loader)

In [0]:
# /////////////// Speckled Images ///////////////
ood_loader = torch.utils.data.DataLoader(ood_data, batch_size=batch_size, shuffle=True,
                                         num_workers=4, pin_memory=True)
speckle = lambda x: torch.clamp(x + x * torch.randn_like(x), 0, 1)
ood_loader.dataset.transform = transforms.Compose([transforms.ToTensor(), speckle, normalize])
speckle_loader = list(ood_loader)

In [0]:
# /////////////// Pixelated Images ///////////////
ood_loader = torch.utils.data.DataLoader(ood_data, batch_size=batch_size, shuffle=True,
                                         num_workers=4, pin_memory=True)
pixelate = lambda x: x.resize((int(32 * 0.2), int(32 * 0.2)), PILImage.BOX).resize((32, 32), PILImage.BOX)
ood_loader.dataset.transform = transforms.Compose([pixelate, transforms.ToTensor(), normalize])
pixelate_loader = list(ood_loader)

In [0]:
# /////////////// RGB Ghosted/Shifted Images ///////////////
ood_loader = torch.utils.data.DataLoader(ood_data, batch_size=batch_size, shuffle=True,
                                         num_workers=4, pin_memory=True)
rgb_shift = lambda x: torch.cat((x[1:2].index_select(2, torch.LongTensor([i for i in range(32 - 1, -1, -1)])),
                                 x[2:, :, :], x[0:1, :, :]), 0)
ood_loader.dataset.transform = transforms.Compose([transforms.ToTensor(), rgb_shift, normalize])
rgb_shift_loader = list(ood_loader)

In [0]:
# /////////////// Inverted Images ///////////////
ood_loader = torch.utils.data.DataLoader(ood_data, batch_size=batch_size, shuffle=True,
                                         num_workers=4, pin_memory=True)
# not done on all channels to make image ood with higher probability
invert = lambda x: torch.cat((x[0:1, :, :], 1 - x[1:2, :, ], 1 - x[2:, :, :],), 0)
ood_loader.dataset.transform = transforms.Compose([transforms.ToTensor(), invert, normalize])
invert_loader = list(ood_loader)

## Code for Detecting OODs

<b> Extract predictions for train and test data </b>

In [0]:
train_preds = []
train_confs = []
train_logits = []
for idx in range(0,len(data_train),128):
    batch = torch.squeeze(torch.stack([x[0] for x in data_train[idx:idx+128]]),dim=1).cuda()
    
    logits = torch_model(batch)
    confs = F.softmax(logits,dim=1).cpu().detach().numpy()
    preds = np.argmax(confs,axis=1)
    logits = (logits.cpu().detach().numpy())#**2)#.sum(axis=1)

    train_confs.extend(np.max(confs,axis=1))    
    train_preds.extend(preds)
    train_logits.extend(logits)
print("Done")

test_preds = []
test_confs = []
test_logits = []

for idx in range(0,len(data),128):
    batch = torch.squeeze(torch.stack([x[0] for x in data[idx:idx+128]]),dim=1).cuda()
    
    logits = torch_model(batch)
    confs = F.softmax(logits,dim=1).cpu().detach().numpy()
    preds = np.argmax(confs,axis=1)
    logits = (logits.cpu().detach().numpy())#**2)#.sum(axis=1)

    test_confs.extend(np.max(confs,axis=1))    
    test_preds.extend(preds)
    test_logits.extend(logits)
print("Done")

Done
Done


<b> Code for detecting OODs by identifying anomalies in correlations </b>

In [0]:
import zero_shot_calculate_log as callog
# torch_model = torch_model.module
def detect(all_test_deviations,all_ood_deviations, verbose=True):
    average_results = {}
    for i in range(1,11):
        random.seed(i)
        
        validation_indices = random.sample(range(len(all_test_deviations)),int(0.1*len(all_test_deviations)))
        test_indices = sorted(list(set(range(len(all_test_deviations)))-set(validation_indices)))

        validation = all_test_deviations[validation_indices]
        test_deviations = all_test_deviations[test_indices]

        t95 = validation.mean(axis=0)+10**-7

        test_deviations = (test_deviations/t95[np.newaxis,:]).sum(axis=1)
        ood_deviations = (all_ood_deviations/t95[np.newaxis,:]).sum(axis=1)
        
        results = callog.compute_metric(-test_deviations,-ood_deviations)
        for m in results:
            average_results[m] = average_results.get(m,0)+results[m]
    
    for m in average_results:
        average_results[m] /= i
    if verbose:
        callog.print_results(average_results)
    return average_results

def Analyse(dataset,POWERS=[10]):
    ood_preds = []
    ood_confs = []
    ood = dataset
    
    for idx in range(0,len(ood),128):
        batch = torch.squeeze(torch.stack([x[0] for x in ood[idx:idx+128]]),dim=1).cuda()
        logits = torch_model(batch)
        confs = F.softmax(logits,dim=1).cpu().detach().numpy()
        preds = np.argmax(confs,axis=1)

        ood_confs.extend(np.max(confs,axis=1))
        ood_preds.extend(preds)   
    print("Done")
    
    def compute_deviation(PRED):
        train_indices = np.where(np.array(train_preds)==PRED)[0]
        test_indices = np.where(np.array(test_preds)==PRED)[0]
        ood_indices = np.where(np.array(ood_preds)==PRED)[0]

        train_PRED = torch.squeeze(torch.stack([data_train[i][0] for i in train_indices]),dim=1)
        test_PRED = torch.squeeze(torch.stack([data[i][0] for i in test_indices]),dim=1)
        if len(ood_indices)!=0:
            ood_PRED = torch.squeeze(torch.stack([ood[i][0] for i in ood_indices]),dim=1)

        test_confs_PRED = np.array([test_confs[i] for i in test_indices])
        if len(ood_indices)!=0:
            ood_confs_PRED =  np.array([ood_confs[i] for i in ood_indices])

        mins,maxs = torch_model.get_min_max(train_PRED,power=POWERS)

        test_deviations = torch_model.get_deviations(test_PRED,power=POWERS,mins=mins,maxs=maxs)/test_confs_PRED[:,np.newaxis]

        if len(ood_indices)!=0:
            ood_deviations = torch_model.get_deviations(ood_PRED,power=POWERS,mins=mins,maxs=maxs)/ood_confs_PRED[:,np.newaxis]
        else:
            ood_deviations = None

        return test_deviations,ood_deviations
    
    CLASSES = range(10)
    
    all_test_deviations = None
    all_ood_deviations = None

    
    for PRED in tqdm(CLASSES):
        test_deviations,ood_deviations = compute_deviation(PRED)
        
        if all_test_deviations is not None:
            all_test_deviations = np.concatenate([all_test_deviations,test_deviations],axis=0) 
        else:
            all_test_deviations = test_deviations
        if ood_deviations is not None:
            if all_ood_deviations is not None:
                all_ood_deviations = np.concatenate([all_ood_deviations,ood_deviations],axis=0)
            else:
                all_ood_deviations = ood_deviations
    
    
    average_results = detect(all_test_deviations,all_ood_deviations)
    return average_results, all_test_deviations, all_ood_deviations


<center><h1> Results </h1></center>

## Combination of Pre- and Post-Training Methods

### Validation

In [0]:
print("Uniform Noise")
uniform_results=Analyse(uniform_loader,POWERS=range(1,11))
print()
print("Arithmetic Mean of Images")
ami_results =Analyse(ami_loader,POWERS=range(1,11))
print()
print("Geometric Mean of Images")
gmi_results =Analyse(gmi_loader,POWERS=range(1,11))
print()
print("Jigsaw Images")
jigsaw_results =Analyse(jigsaw_loader,POWERS=range(1,11))
print()
print("Speckle Images")
speckle_results =Analyse(speckle_loader,POWERS=range(1,11))
print()
print("Pixelated Images")
pixelate_results =Analyse(pixelate_loader,POWERS=range(1,11))
print()
print("RGB Ghosted/Shifted Images")
rgb_shift_results =Analyse(rgb_shift_loader,POWERS=range(1,11))
print()
print("Inverted Images\n")
inverted_results =Analyse(invert_loader,POWERS=range(1,11))

Uniform Noise
Done



 TNR    AUROC  DTACC  AUIN   AUOUT 
 100.000 99.992 99.993 99.993 99.787

Arithmetic Mean of Images
Done



 TNR    AUROC  DTACC  AUIN   AUOUT 
 10.384 52.831 54.749 45.695 58.128

Geometric Mean of Images
Done



 TNR    AUROC  DTACC  AUIN   AUOUT 
  0.000  0.000 50.000 28.683 32.751

Jigsaw Images
Done



 TNR    AUROC  DTACC  AUIN   AUOUT 
 62.997 89.771 82.443 84.538 90.655

Speckle Images
Done



 TNR    AUROC  DTACC  AUIN   AUOUT 
 100.000 100.000 99.989 99.994 99.995

Pixelated Images
Done



 TNR    AUROC  DTACC  AUIN   AUOUT 
 100.000 99.974 99.763 99.976 99.931

RGB Ghosted/Shifted Images
Done



 TNR    AUROC  DTACC  AUIN   AUOUT 
 75.126 94.288 87.035 92.043 94.904

Inverted Images

Done



 TNR    AUROC  DTACC  AUIN   AUOUT 
 99.262 99.752 97.846 99.754 99.714


### Testing

In [0]:
print("LSUN_R")
lsun_results=Analyse(lsun,POWERS=range(1,11))

#print("iSUN_R")
#isun_results=Analyse(isun,POWERS=range(1,11)) 

print("TinyImageNet_R")
tinyimagenet_results=Analyse(tinyimagenet,POWERS=range(1,11))

print("SVHN")
svhn_results=Analyse(svhn,POWERS=range(1,11))

LSUN_R
Done



 TNR    AUROC  DTACC  AUIN   AUOUT 
 99.817 99.925 98.979 99.919 99.915
TinyImageNet_R
Done



 TNR    AUROC  DTACC  AUIN   AUOUT 
 99.326 99.771 98.267 99.727 99.792
SVHN
Done



 TNR    AUROC  DTACC  AUIN   AUOUT 
 98.483 99.603 97.438 98.645 99.862
